# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [24]:
from xml.etree import ElementTree as ET
import urllib

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [54]:
document_tree = ET.parse( './data/mondial_database_less.xml' )
import pandas as pd

In [52]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [53]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [132]:
#Question 1: 10 Countries with the lowest infant mortality rates

import pandas as pd
document = ET.parse( './data/mondial_database.xml' )

im_df = [] #create a temporary list

for element in document.iterfind('country'): #for all elements named country, we would like to...
    if (element.find('infant_mortality')) != None:
        name = element.find('name').text
        name = str(name) #convert text to string
        infant_mortality = element.find('infant_mortality').text
        infant_mortality = float(infant_mortality) #convert the text to float
        im_df.append([name,infant_mortality]) 
    else :
        continue    
im_df = pd.DataFrame(im_df, columns = ['Country','Infant Mortality']) #convert list to data frame
im_df.sort_values('Infant Mortality')[0:10]



,Country,Infant Mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [192]:
#Question 2: 10 cities with the largest population

df_population = []

for element in document.iterfind('country'):
    #country = element.find('name').text #included in case we're interested in countries
    #country = str(country) #included in case we're interested in countries
    for subelement in element.iterfind('city'):
        city = subelement.find('name').text
        city = str(city)
        for subelement_1 in subelement.iterfind('population'):
            population = subelement_1.text
            population = float(population)
            df_population.append([city,population])

#df_population = pd.DataFrame(df_population, columns= ['country','city','population'])
df_population = pd.DataFrame(df_population, columns= ['city','population'])
df_population.groupby('city').max().sort_values('population',ascending = False)[0:10]


,population
city,
Seoul,10229262.0
Al Qahirah,8471859.0
Bangkok,7506700.0
Hong Kong,7055071.0
Ho Chi Minh,5968384.0
Singapore,5076700.0
Al Iskandariyah,4123869.0
New Taipei,3939305.0
Busan,3813814.0


In [336]:
#Question 3: 10 ethnic groups with the largest worldwide population - sum of latest estimates over
#all countries

df_ethnicgroup = []
df_population = []

for element in document.iterfind('country'):
    country = element.find('name').text
    country = str(country)
    for pop in element.iterfind('population'):
        population = pop.text
        population = float(population)
        df_population.append([country,population])
    for eg in element.iterfind('ethnicgroup'):
        ethnic_group = eg.text
        ethnic_group_percentage = eg.attrib
        df_ethnicgroup.append([country,ethnic_group,ethnic_group_percentage])
        

df_ethnicgroup = pd.DataFrame(df_ethnicgroup, columns = ['country','ethnic_group','ethnic_group_percentage'])
df_eg = []
for x in range(len(eg_percentage)):
    df_eg.append(float(eg_percentage[x].get('percentage')))                      
df_ethnicgroup['eg_percentage'] = df_eg
del df_ethnicgroup['ethnic_group_percentage']

df_population = pd.DataFrame(df_population, columns = ['country','population'])
df_population = df_population.groupby('country').max()
population_dict = df_population['population'].to_dict()
df_ethnicgroup['population'] = df_ethnicgroup['country'].map(population_dict)
df_ethnicgroup['ethnic_population'] = df_ethnicgroup['eg_percentage']*df_ethnicgroup['population']*.01
df_ethnicgroup = df_ethnicgroup.groupby('ethnic_group').sum().sort_values('ethnic_population', ascending = False)[0:10]
del df_ethnicgroup['eg_percentage']
del df_ethnicgroup['population']
df_ethnicgroup

,ethnic_population
ethnic_group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.949395e+08
African,3.183597e+08
Dravidian,3.027137e+08
Mestizo,1.578553e+08
Bengali,1.467769e+08
Russian,1.368666e+08
Japanese,1.272890e+08


In [419]:
#Question 4a - Name and Country of longest river and highest elevation airport
import pandas as pd

df_river = []

for element in document.iterfind('river'):
    river_length = element.findtext('length')
    river_name = element.find('name').text
    river_name = str(river_name)
    river_country = element.attrib.get('country')
    river_country = str(river_country)
    df_river.append([river_name,river_country,river_length])
    
df_river = pd.DataFrame(df_river,columns=['river name','countries','river length'])
df_river['river length'] = df_river['river length'].astype(float)
df_river = df_river.sort_values('river length', ascending = False).head(1)
df_river



,river name,countries,river length
174,Amazonas,CO BR PE,6448.0


In [420]:
#Question 4b Name and Country of largest lake 

df_lake = []

for element in document.iterfind('lake'):
        lake_name = element.find('name').text
        lake_country = element.get('country')
        lake_area = element.findtext('area')
        df_lake.append([lake_name,lake_country,lake_area])
    
df_lake = pd.DataFrame(df_lake,columns=['lake name','countries','lake size'])
df_lake['lake size'] = df_lake['lake size'].astype(float)
df_lake.sort_values('lake size', ascending = False).head(1)

,lake name,countries,lake size
54,Caspian Sea,R AZ KAZ IR TM,386400.0


In [421]:
#Question 4c Name and Country of highest elevation airport
df_airport = []

for element in document.iterfind('airport'):
        airport_name = element.find('name').text
        airport_country = element.get('country')
        airport_elevation = element.find('elevation').text
        df_airport.append([airport_name, airport_country, airport_elevation])
    
df_airport = pd.DataFrame(df_airport,columns=['airport name','country', 'airport elevation'])
df_airport['airport elevation'] = df_airport['airport elevation'].astype(float)
df_airport.sort_values('airport elevation', ascending = False).head(1)


,airport name,country,airport elevation
80,El Alto Intl,BOL,4063.0
